<a href="https://colab.research.google.com/github/JonathanLehner/Colab-collection/blob/master/python_render_obj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Python 3 with GPU

In [0]:
# ensure the code is as deterministic as possible and allow running it with multiple seeds
# https://pytorch.org/docs/stable/notes/randomness.html
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [0]:
import os
os.environ["PYOPENGL_PLATFORM"] = "egl" #opengl seems to only work with TPU
!PYOPENGL_PLATFORM=egl python -c "from OpenGL import EGL"
print(os.environ['PYOPENGL_PLATFORM']) 


egl


In [0]:
# Install pyrender
!pip install pyrender
!pip install transforms3d
!pip install torch torchvision


In [0]:
import OpenGL.GL as gl
print(gl.glGetString(gl.GL_VERSION))
print(gl.glGetString(gl.GL_VENDOR)) 


None
None


In [0]:
import pyrender
import os
import os.path
import numpy as np
import trimesh
import pyrender
import transforms3d
import OpenGL.GL as gl
import torch
import numpy as np
from matplotlib import pyplot as plt
from scipy.misc import face
from skimage import io
from PIL import Image

!mkdir /content/training_images


mkdir: cannot create directory ‘/content/training_images’: File exists


In [0]:
%cd /content/
# download background images dataset
# !rm ADE20K_2016_07_26.zip 
if os.path.exists('ADE20K_2016_07_26') == False:
  !wget http://groups.csail.mit.edu/vision/datasets/ADE20K/ADE20K_2016_07_26.zip
  !unzip /content/ADE20K_2016_07_26.zip

# collect path names
image_paths = []
num_images = 0
for dirpath, dirnames, filenames in os.walk("/content/ADE20K_2016_07_26"):
    for filename in [f for f in filenames if f.endswith(".jpg")]:
        path = os.path.join(dirpath, filename)
        image_paths.append(path)
        num_images += 1
        
print(image_paths)
print(num_images)

%cd /content/
# download hand model
if os.path.exists('/content/hands') == False:
  !mkdir /content/hands
  %cd /content/hands/
  !wget https://github.com/JonathanLehner/Colab-collection/releases/download/MANO/hand_mesh_0.obj_corrected.obj
  !wget https://github.com/JonathanLehner/Colab-collection/releases/download/MANO/hand_mesh_1.obj_corrected.obj
  !wget https://github.com/JonathanLehner/Colab-collection/releases/download/MANO/hand_mesh_2.obj_corrected.obj
  !wget https://github.com/JonathanLehner/Colab-collection/releases/download/MANO/hand_mesh_3.obj_corrected.obj
  !wget https://github.com/JonathanLehner/Colab-collection/releases/download/MANO/hand_mesh_4.obj_corrected.obj
  !wget https://github.com/JonathanLehner/Colab-collection/releases/download/MANO/hand_mesh_5.obj_corrected.obj
  %cd /content/

/content
['/content/ADE20K_2016_07_26/images/training/z/zen_garden/ADE_train_00020196.jpg', '/content/ADE20K_2016_07_26/images/training/z/zen_garden/ADE_train_00020193.jpg', '/content/ADE20K_2016_07_26/images/training/z/zen_garden/ADE_train_00020191.jpg', '/content/ADE20K_2016_07_26/images/training/z/zen_garden/ADE_train_00020189.jpg', '/content/ADE20K_2016_07_26/images/training/z/zen_garden/ADE_train_00020194.jpg', '/content/ADE20K_2016_07_26/images/training/z/zen_garden/ADE_train_00020192.jpg', '/content/ADE20K_2016_07_26/images/training/z/zen_garden/ADE_train_00020190.jpg', '/content/ADE20K_2016_07_26/images/training/z/zen_garden/ADE_train_00020195.jpg', '/content/ADE20K_2016_07_26/images/training/z/zen_garden/ADE_train_00020197.jpg', '/content/ADE20K_2016_07_26/images/training/z/zoo/ADE_train_00020204.jpg', '/content/ADE20K_2016_07_26/images/training/z/zoo/ADE_train_00020207.jpg', '/content/ADE20K_2016_07_26/images/training/z/zoo/ADE_train_00020206.jpg', '/content/ADE20K_2016_07_26

In [0]:
# collect hand meshes
hand_paths = []
num_hands = 0
for dirpath, dirnames, filenames in os.walk("/content/hands"):
    print(filenames)
    for filename in filenames:
        if(filename.endswith("_corrected.obj")): # only use corrected hand meshes
             path = os.path.join(dirpath, filename)
             hand_paths.append(path)        
             num_hands += 1
        
print(hand_paths)
print(num_hands)

['hand_mesh_2.obj_corrected.obj', 'hand_mesh_3.obj_corrected.obj', 'hand_mesh_1.obj_corrected.obj', 'hand_mesh_4.obj_corrected.obj', 'hand_mesh_5.obj_corrected.obj', 'hand_mesh_0.obj_corrected.obj']
['/content/hands/hand_mesh_2.obj_corrected.obj', '/content/hands/hand_mesh_3.obj_corrected.obj', '/content/hands/hand_mesh_1.obj_corrected.obj', '/content/hands/hand_mesh_4.obj_corrected.obj', '/content/hands/hand_mesh_5.obj_corrected.obj', '/content/hands/hand_mesh_0.obj_corrected.obj']
6


In [0]:
# generate random values for 100,000 training images
# we take 100 different meshes (shape and pose)
# with each 100 different rotations
# and 10 different background images. In total: 100*100*10 images

backgrounds = torch.randint(low=0, high=num_images-1, size=(100,100,10), out=None, dtype=None, layout=torch.strided, device=None, requires_grad=False) 
# print(backgrounds)
rotations = torch.rand((100,100,3), out=None, dtype=None, layout=torch.strided, device=None, requires_grad=False) * 2 * np.pi

In [0]:

# translations
T = [[1.0, 0.0, 0.0, 0.0],
     [0.0, 1.0, 0.0, 0.0],
     [0.0, 0.0, 1.0, 200.0],
     [0.0, 0.0, 0.0, 1.0]]
Z = [1.0, 1.0, 1.0] # zooms

scene = pyrender.Scene(ambient_light=np.array([1.7, 1.7, 1.7, 1.0]), bg_color=[1.0, 1.0, 1.0, 0.0])
# Set up the camera -- z-axis away from the scene, x-axis right, y-axis up
camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0)
cam_in_scene = scene.add(camera)
    
for num_hand, hand_path in enumerate(hand_paths):
    print(hand_path)
    
    hand_trimesh = trimesh.load(hand_path)
    
    vertex_colors = np.random.uniform(size=(779, 3)) # hand_trimesh.vertices.shape
    hand_trimesh.visual.vertex_colors = vertex_colors

    hand_mesh = pyrender.Mesh.from_trimesh(hand_trimesh, smooth=False) #, material
    hand_node = pyrender.Node(mesh=hand_mesh)

    scene.add_node(hand_node)

    for num_rot, rotation_values in enumerate(rotations[0]): 

        rot_x = rotation_values[0]
        rot_y = rotation_values[1]
        rot_z = rotation_values[2]
        rotation = transforms3d.euler.euler2mat(rot_x, rot_y, rot_z, axes='sxyz')
        B = transforms3d.affines.compose([45.0, 0.0, 0.0], rotation, Z) # applies translation last
        camera_pose = np.matmul(B, T)

        scene.set_pose(cam_in_scene, pose=camera_pose)

        # Set up the light -- a single spot light in the same spot as the camera
        light = pyrender.SpotLight(color=np.ones(3), intensity=3.0,
                                       innerConeAngle=np.pi/16.0)
        scene.add(light, pose=camera_pose)

        # Render the scene
        r = pyrender.OffscreenRenderer(viewport_width=640*10, viewport_height=480*10)
        flags = pyrender.RenderFlags.RGBA | pyrender.RenderFlags.OFFSCREEN
        color, depth = r.render(scene, flags=flags)
        r.delete()

        # show rendered hand in front of a random background image
        # source: https://stackoverflow.com/questions/34495817/plotting-a-background-image-to-a-matplotlib-graph

        for num_bg, background in enumerate(backgrounds[0][num_rot]):
            fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,10) )
            plt.axis('on')

            print(background)

            image_path = image_paths[background]
            print(image_path)
            im = Image.open(image_path)

            background = ax.imshow(im, cmap=plt.cm.gray, extent=[0, 1, 0, 1])
            image = ax.imshow(color, extent=[0, 1, 0, 1])

            %cd /content/
            plt.savefig('training_images/testplot.png')

            plt.show()
            
            
    scene.remove_node(hand_node)

/content/hands/hand_mesh_2.obj_corrected.obj


GLError: ignored

In [0]:
# save data to csv file with Pandas